In [1]:
import re

def constant_propagation(TAC):
    for i in range(len(TAC)):
        expression = TAC[i].split('=')
        var = expression[0].strip()
        value = expression[1].strip()

        # Check if the value is a constant (integer or float)
        if is_constant(value):
            for j in range(len(TAC)):
                parts = TAC[j].split('=')
                variable_name = parts[0].strip()
                expression = parts[1].strip()
                
                # Replace variable with constant value in subsequent expressions
                if var in expression:
                    TAC[j] = f"{variable_name} = {expression.replace(var, value)}"
    return TAC

def constant_folding(TAC):
    for i in range(len(TAC)):
        expression = TAC[i].split('=')
        var = expression[0].strip()
        value = expression[1].strip()

        # Check if the expression on the right-hand side contains arithmetic operation
        if '+' in value or '-' in value or '*' in value or '/' in value:
            try:
                result = eval(value)
                TAC[i] = f"{var} = {result}"
            except:
                pass
    return TAC

def copy_propagation(TAC):
    replacements = {}  # Dictionary to store variable replacements

    for i, instruction in enumerate(TAC):
        parts = instruction.split('=')
        var = parts[0].strip()
        value = parts[1].strip()

        # Check if the value is assigned from another variable
        if len(parts) == 2 and value in replacements:
            TAC[i] = f"{var} = {replacements[value]}"
        else:
            replacements[var] = value

    return TAC

def common_subexpression_elimination(TAC):
    expressions = {}  # Dictionary to store expressions and their corresponding variables

    for i, instruction in enumerate(TAC):
        parts = instruction.split('=')
        var = parts[0].strip()
        expression = parts[1].strip()

        # Check if the expression is already encountered
        if expression in expressions:
            if expressions[expression] != var:
                print(f"Common Subexpression Elimination: Replacing {var} with {expressions[expression]} for expression {expression}")
                TAC[i] = f"{var} = {expressions[expression]}"
        else:
            expressions[expression] = var

    return TAC

def dead_code_elimination(TAC):
    used_variables = set()
    used_constants = set()
    defined_variables = set()

    # Identify defined variables and constants
    for instruction in TAC:
        parts = instruction.split('=')
        var = parts[0].strip()
        value = parts[1].strip()

        defined_variables.add(var)

        # Check if the value is a constant
        if is_constant(value):
            used_constants.add(value)

    # Identify used variables
    for instruction in TAC:
        parts = instruction.split('=')
        var = parts[0].strip()
        value = parts[1].strip()

        # Add variable to used variables
        if not is_constant(value):
            used_variables.add(var)
        else:
            if value in defined_variables:
                used_variables.add(value)

    # Identify dead code (unused variables and constants)
    dead_code = [instruction for instruction in TAC if instruction.split('=')[0].strip() not in used_variables]

    # Remove dead code from TAC
    TAC = [instruction for instruction in TAC if instruction not in dead_code]

    return TAC

def is_constant(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def optimize_TAC(TAC):
    print(f"Initial Three Address Code:")
    for line in TAC:
        print(line)

    print("\nOptimization Steps:")
    
    # Flags to track changes in each optimization step
    changed_propagation = False
    changed_copy = False
    changed_folding = False
    changed_subexpression = False

    changed = True  # Flag to track overall changes

    while changed:
        changed = False  # Reset the overall flag
        
        # Apply optimizations
        original_TAC = TAC.copy()  # Make a copy to check for changes
        
        # Constant Propagation
        TAC = constant_propagation(TAC)
        if TAC != original_TAC:
            changed_propagation = True
            changed = True
            print("\nConstant Propagation:")
            for line in TAC:
                print(line)
        
        # Copy Propagation
        original_TAC = TAC.copy()  # Make a copy to check for changes
        TAC = copy_propagation(TAC)
        if TAC != original_TAC:
            changed_copy = True
            changed = True
            print("\nCopy Propagation:")
            for line in TAC:
                print(line)
        
        # Constant Folding
        original_TAC = TAC.copy()  # Make a copy to check for changes
        TAC = constant_folding(TAC)
        if TAC != original_TAC:
            changed_folding = True
            changed = True
            print("\nConstant Folding:")
            for line in TAC:
                print(line)
        
        # Common Subexpression Elimination
        original_TAC = TAC.copy()  # Make a copy to check for changes
        TAC = common_subexpression_elimination(TAC)
        if TAC != original_TAC:
            changed_subexpression = True
            changed = True
            print("\nCommon Subexpression Elimination:")
            for line in TAC:
                print(line)

    # Dead Code Elimination (applied after all other optimizations)
    print("\nDead Code Elimination:")
    TAC = dead_code_elimination(TAC)
    for line in TAC:
        print(line)

In [2]:
# Given Three Address Code (TAC)
TAC = [
    "a = 2",
    "b = x * x",
    "c = x",
    "d = a + 5",
    "e = b + c",
    "f = c * c",
    "g = d + e",
    "h = e * f"    
]

# Test case
print("Test Case:")
optimize_TAC(TAC)

Test Case:
Initial Three Address Code:
a = 2
b = x * x
c = x
d = a + 5
e = b + c
f = c * c
g = d + e
h = e * f

Optimization Steps:

Constant Propagation:
a = 2
b = x * x
c = x
d = 2 + 5
e = b + c
f = c * c
g = d + e
h = e * f

Constant Folding:
a = 2
b = x * x
c = x
d = 7
e = b + c
f = c * c
g = d + e
h = e * f

Constant Propagation:
a = 2
b = x * x
c = x
d = 7
e = b + c
f = c * c
g = 7 + e
h = e * f

Dead Code Elimination:
b = x * x
c = x
e = b + c
f = c * c
g = 7 + e
h = e * f
